|<h2>Course:</h2>|<h1><a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">A deep understanding of AI language model mechanisms</a></h1>|
|-|:-:|
|<h2>Part 5:</h2>|<h1>Observation (non-causal) mech interp<h1>|
|<h2>Section:</h2>|<h1>Investigating neurons and dimensions<h1>|
|<h2>Lecture:</h2>|<h1><b>CodeChallenge HELPER: Negation tuning in attention neurons<b></h1>|

<br>

<h5><b>Teacher:</b> Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h5>
<h5><b>Course URL:</b> <a href="https://udemy.com/course/dullms_x/?couponCode=202508" target="_blank">udemy.com/course/dullms_x/?couponCode=202508</a></h5>
<i>Using the code without the course may lead to confusion or errors.</i>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.gridspec import GridSpec

import statsmodels.api as sm

import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, GPT2Tokenizer

import requests

# vector plots
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# Exercise 1a: Import the model and implant attention hooks

In [ ]:
# load GPT2 model and tokenizer
model = AutoModelForCausalLM.from_pretrained('gpt2-large')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# use GPU

In [ ]:
# hooks
activations = {}

def implant_hook(layer_number):
  def hook(module, input, output):
    activations
  return hook


In [ ]:
nneurons =

# Exercise 1b: Get text, find negations and affirmations

In [ ]:
# https://gutenberg.org/ebooks/32154
text = requests.get('https://gutenberg.org/cache/epub/32154/pg32154.txt').text
tokens =
num_tokens =
print(f'There are {} tokens, of which {} are unique.')

In [ ]:
# context window size (tokens before and after each target)
context_pre =
context_pst =

In [ ]:
# list of negation words to match exactly
negation_words = []

# initialize vector
isNegation = np.zeros(num_tokens,dtype=

# loop over all tokens
for ti in range(context_pre,num_tokens-context_pst):

  # current token
  currtok =

  # token contains a 't contraction
  condA =

  # word need to match completely (c.f., not->noted, nor->enormous)
  condB =

  # next token starts with a space (c.f., not->connotative)
  condC =

  # test
  if
    isNegation[ti] = 1


# count the targets
numNegationTokens =
negationsIdx =

In [ ]:
# list of affirmation words to match exactly
affirmation_words = ['agree','always','allow','can','certainly','could','definitely','may','might','shall','should']

# initialize vector
isAffirmation =

# loop over all tokens
for ti in

  # current token
  currtok =

  # next token can't be 'not'
  condA =

  # word need to match completely (c.f., not->noted, nor->enormous)
  condB =

  # next token starts with a space (c.f., not->connotative)
  condC =

  # test
  if condA and condB and condC:
    isAffirmation[ti] = 1

# count the number of target tokens
numAffirmationTokens = sum(isAffirmation)
affirmationsIdx = np.where(isAffirmation)[0]

# Exercise 1c: Create batches and get activations

In [ ]:
# create batches
batch_negations =
batch_affirmations =


# negation sequences
for b in range(numNegationTokens):
  tokenLoc =
  batch_negations[b,:] = tokens

# affirmation sequences
for b in range(numAffirmationTokens):


#
print('Shape of negations batch:',batch_negations.shape)
print('Shape of affirmations batch:',batch_affirmations.shape)

In [ ]:
# process the target (negation) tokens
with torch.no_grad():
  model(batch_negations.to(device))

# copy the activations



### repeat for affirmations tokens


In [ ]:
print(affirmations_activations.keys(),'\n')

affirmations_activations['attn_5'].shape

# Exercise 2: Laminar profile of classification

In [ ]:
# vector of category labels
category_labels = np.hstack

In [ ]:
# initialize matrix to store the classifier results
pvalues  =
betas    =
accuracy =


# loop over layers
for layeri

  # loop over neurons for per-neuron analysis
  for neuroni in range(nneurons):
    # vectorize the activations over batches
    targs = negations_activations
    comps = affirmations_activations

    # build and run the model
    try: # sometimes crashes for linear-algebra reasons
      result = sm.Logit

      # extract the results (p-value, beta, and accuracy)
      pvalues[layeri,neuroni]  =
      betas[layeri,neuroni]    =
      accuracy[layeri,neuroni] =

    except: pass

  print(f'Finished layer {layeri+1:2}/{model.config.n_layer}')

In [ ]:
# create two masks
pvalue_mask =
posbet_mask =

# get accuracy only from masked neurons
# gratuitously confusingly, np.ma.masked_where() actually keeps the False values, and masks *out* True values
masked_accuracy = np.ma.masked_where


# split the accuracy matrix into the Q/K/V matrices
q,k,v = torch.tensor(masked_accuracy).split


# make the plot
_,axs = plt.subplots(1,2,figsize=(12,4))

axs[0].plot(,'kH',markerfacecolor=[.7,.7,.7],markersize=9)
axs[0].set(xlabel='Layer',ylabel='Percent significant tests (%)',title='Laminar profile of significance')

axs[1].plot(,'ko',markerfacecolor=[.9,.7,.7],markersize=9,label='Q')
axs[1].plot(,'ks',markerfacecolor=[.7,.9,.7],markersize=9,label='K')
axs[1].plot(,'k^',markerfacecolor=[.7,.7,.9],markersize=9,label='V')
axs[1].legend()


plt.show()